In [ ]:
import geemap
import ee

In [ ]:
Map = geemap.Map(center=[29.66,116], zoom=10)
#添加面shp（文件名不能有“_”和中文）
states_shp = r'F:\file\work\师兄\extract-values\Pt\PtRiceProj.shp'
jx = geemap.shp_to_ee(states_shp)   #必须先安装pip install pyshp #filter_polygons(states_shp)
Map.addLayer(jx, {}, 'JX')

In [ ]:
Map

In [ ]:
inpath = r'F:\Pt'
crop_points = inpath + os.sep + 'PtRiceProj.shp'
p_shp = geemap.shp_to_ee(crop_points)
Map.addLayer(p_shp, {'color': 'FF0000'}, 'p_shp')

In [ ]:
p_shp.getInfo()

In [ ]:
def cloudfunction_L8(image):
    # image = ee.Algorithms.Landsat.simpleCloudScore(image)
    image = image.updateMask(image.select("cloud").lte(20));
    return image;

l8bands = ['B2','B3','B4','B5','B6','B7']
#添加landsat8
l8m = ee.ImageCollection('LANDSAT/LC08/C01/T1_RT_TOA')\
                .filterDate('2017-05-01', '2017-08-01')\
                .filterBounds(jx)\
                .map(ee.Algorithms.Landsat.simpleCloudScore)\
                .map(cloudfunction_L8)\
                .mosaic()\
                .clip(jx)\
                .select(l8bands)


In [ ]:
import pandas as pd
# 输出参数
# outpath = r'F:\JJUDATA\ExctraValue'
outFile = r'F:\Ext03211119.csv'
geemap.extract_values_to_points(p_shp,l8m, outFile, scale=30)